# Imports

In [606]:
import numpy as np
import pandas as pd
from pylab import rcParams
import string
import re
import math
from matplotlib import rc
from collections import Counter, defaultdict
from math import sqrt
from itertools import product
from numba import jit,njit, prange

# Preparing Data

In [607]:
pauwel = pd.read_csv("pauwel-dataset.txt", sep='\t', header=0)
pauwel

,abdominal cramps,abdominal distention,abdominal pain,malformations,spontaneous abortion,missed abortion,abscess,acanthosis nigricans,acidosis,renal tubular acidosis,...,drug dependence,diverticulosis,prostatic hypertrophy,allergic reaction,dysphonia,eosinophilic pneumonia,retinal vein thrombosis,renal insufficiency,glioblastoma multiforme,portal cirrhosis
carnitine,1,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
GABA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
delta-aminolevulinic acid,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
leucovorin,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PGE2,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pimecrolimus,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
auranofin,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cefditoren,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
nitroprusside,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [608]:
mizutani = pd.read_csv("mizutani-dataset.txt", sep='\t', header=0)
mizutani # drug names are coded with pubchem ID

,abdominal.cramps,abdominal.distention,abdominal.pain,malformations,spontaneous.abortion,missed.abortion,abscess,acanthosis.nigricans,acidosis,renal.tubular.acidosis,...,vitamin.deficiency,drug.dependence,diverticulosis,prostatic.hypertrophy,allergic.reaction,dysphonia,eosinophilic.pneumonia,retinal.vein.thrombosis,renal.insufficiency,glioblastoma.multiforme
85,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
119,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
137,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
143,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
158,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6398525,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6398970,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
6447131,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
6918453,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [609]:
liu = pd.read_csv("liu-dataset.csv", sep=',', header=None)
liu

,0,1,2,3,4,5,6,7,8,9,...,1375,1376,1377,1378,1379,1380,1381,1382,1383,1384
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
827,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
828,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
829,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
830,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [610]:
# Check number of missing values
new_pauwel = pauwel.fillna(999).values
num_missing_pauwel = np.argwhere(new_pauwel == 999)
print("The number of missing values in pauwel's dataset:" ,len(num_missing_pauwel))

The number of missing values in pauwel's dataset: 0


In [611]:
new_mizutani = mizutani.fillna(999).values
num_missing_mizutani = np.argwhere(new_mizutani == 999)
print("The number of missing values in mizutani's dataset:",len(num_missing_mizutani))

new_liu = liu.fillna(999).values
num_missing_liu = np.argwhere(new_liu == 999)
print("The number of missing values in pauwel's dataset:" ,len(num_missing_liu))


The number of missing values in mizutani's dataset: 0
The number of missing values in pauwel's dataset: 0


In [612]:
def density(df):
    density = float(len(np.nonzero(df.to_numpy())[0]))
    density /= (df.shape[0]*df.shape[1])
    density *= 100
    return density

In [613]:
# Check the density of the pauwel dataset
density = float(len(np.nonzero(pauwel.to_numpy())[0]))
print(density)
density /= (pauwel.shape[0]*pauwel.shape[1])
density *= 100
print('{:.2f}%'.format(density))

61102.0
4.97%


In [614]:
# Check the density of the mizutani dataset
density = float(len(np.nonzero(mizutani.to_numpy())[0]))
print(density)
density /= (mizutani.shape[0]*mizutani.shape[1])
density *= 100
print('{:.2f}%'.format(density))

49051.0
5.57%


In [615]:
# Check the density of the liu dataset
density = float(len(np.nonzero(liu.to_numpy())[0]))
print(density)
density /= (liu.shape[0]*liu.shape[1])
density *= 100
print('{:.2f}%'.format(density))

59205.0
5.14%


# Training

## Model Implementation

### Original (Non-Numba)

In [616]:
# class SGD_Recommender:
    
#     def __init__(self, k, lmbda, n_iter=350, learn_rate=0.005, tolerance=1e-04):
#         """Set the parameters for SGD"""
        
#         self.k=k
#         self.lmbda=lmbda
#         self.n_iter=n_iter
#         self.learn_rate=learn_rate
#         self.tolerance=tolerance
        
#     def predictions(self, U: np.ndarray, V: np.ndarray) -> np.ndarray:
#         """Return dot product of the matrices U and V."""
        
#         return np.dot(U, V.T)
    
#     def check_convergence(self, U_old: np.ndarray, U_curr: np.ndarray, V_old: np.ndarray, V_curr: np.ndarray) -> bool:
#         """Check if matrices have reached convergence."""
        
#         # subtract old and current matrices and see if each value in result matrix is less than tolerance 
#         U_converged = np.all(np.abs(np.subtract(U_old,U_curr)) <= self.tolerance)
#         V_converged = np.all(np.abs(np.subtract(V_old,V_curr)) <= self.tolerance)
#         return U_converged and V_converged
        
#     def fit(self, train: pd.DataFrame, dataset: pd.DataFrame) -> None:
#         """Train the SGD model.
        
#         Args:
#             train (pd.DataFrame): The training set
#             dataset (pd.DataFrame): The dataset
#         Returns: 
#             None        
#         """     
        
#         m, n = dataset.shape
        
#         # Initialize the low rank matrices U and V with values from the normal distribution N(0,0.01)
#         mu, sigma = 0, 0.1
#         self.U = np.random.normal(loc=mu, scale=sigma, size=(m, self.k))
#         self.V = np.random.normal(loc=mu, scale=sigma, size=(n, self.k))
        
#         # Get the indices of known drug-se associations in the training set from the dataset
#         drugs, side_fx = train.values.nonzero()
#         drug_names = train.index[drugs]
#         side_fx_names = train.columns[side_fx]
#         drug_index = [dataset.index.get_loc(x) for x in drug_names]
#         side_fx_index = [dataset.columns.get_loc(x) for x in side_fx_names]
        
#         drug_se = list(zip(drug_index, side_fx_index))
#         rng = np.random.default_rng()
        
#         # Start of training
#         for epoch in range(self.n_iter):
#             rng.shuffle(drug_se) # Shuffle in place
#             U_old = self.U.copy()
#             V_old = self.V.copy()
            
#             for (drug, se) in drug_se:
#                 error = dataset.iloc[drug,se]-self.predictions(self.U[drug,:], self.V[se,:])
#                 temp_u = self.U[drug,:] + self.learn_rate*(error*self.V[se,:] - self.lmbda*self.U[drug,:])
#                 temp_v = self.V[se,:] + self.learn_rate*(error*self.U[drug,:] - self.lmbda*self.V[se,:])
#                 self.U[drug,:] = temp_u 
#                 self.V[se,:] = temp_v
            
#             if self.check_convergence(U_old, self.U, V_old, self.V):
#                 break
            
#     def predict(self) -> np.ndarray:
#         """Predict the entire drug-side effect matrix values."""
        
#         return self.predictions(self.U, self.V)

### Numba Implementation

In [617]:
  
# def __init__(k, lmbda, n_iter=350, learn_rate=0.005, tolerance=1e-04):
#     """Set the parameters for SGD"""
        
#     self.k=k
#     self.lmbda=lmbda
#     self.n_iter=n_iter
#     self.learn_rate=learn_rate
#     self.tolerance=tolerance
        
@njit
def predictions(U: np.ndarray, V: np.ndarray) -> np.ndarray:
    """Return dot product of the matrices U and V."""
    return np.dot(U, V.T)

@njit
def check_convergence(U_old: np.ndarray, U_curr: np.ndarray, V_old: np.ndarray, 
            V_curr: np.ndarray, tolerance:float=1e-06) -> bool:
    """Check if matrices have reached convergence."""

    # subtract old and current matrices and see if each value in result matrix is less than tolerance 
    U_converged = np.all(np.abs(np.subtract(U_old,U_curr)) <= tolerance)
    V_converged = np.all(np.abs(np.subtract(V_old,V_curr)) <= tolerance)
    return U_converged and V_converged   
    
@njit
def fit(drug_index: np.ndarray, side_fx_index: np.ndarray, dataset: np.ndarray, 
        k: float, lmbda: float, max_iter:int=350, learn_rate:float=0.005):
    """Train the SGD model.

    Args:
        drug_index-index (np.ndarray): Row indices of the non zero values in train set.
        drug-side_fx_index (np.ndarray): Column indices of the non zero values in train set
        dataset (np.ndarray): The dataset
        k (float): Number of latent features
        lmbda (float): Regularization term.
        max_iter (int): Max number of iterations
        learn_rate (float): Learning rate
    Returns: 
        U, V (np.ndarray): The low rank matrix representation of the drug-side effect matrix.        
    """     
    # print("We're alright here")

    m, n = dataset.shape
    # print(m,n)
    # Initialize the low rank matrices U and V with values from the normal distribution N(0,0.01)
    mu, sigma = 0, 0.1
    U = np.random.normal(loc=mu, scale=sigma, size=(m, k))
    V = np.random.normal(loc=mu, scale=sigma, size=(n, k))
    # print("We're alright here2")

    
    drug_se = np.array(list(zip(drug_index,side_fx_index)))

    # print("We're alright here3")
    # Start of training
    for epoch in range(max_iter):
        # print("We're alright here4")

        np.random.shuffle(drug_se) # Shuffle in place
        U_old = U.copy()
        V_old = V.copy()
        # print("U's shape: ", U_old.shape)
        # print("V's shape: ", V_old.shape)
        # Learn from the known  associations in the training set (drug_se)
        for (drug, se) in drug_se:
            error = dataset[drug,se] - predictions(U[drug,:], V[se,:])
            temp_u = U[drug,:] + learn_rate*(error*V[se,:] - lmbda*U[drug,:])
            temp_v = V[se,:] + learn_rate*(error*U[drug,:] - lmbda*V[se,:])
            U[drug,:] = temp_u 
            V[se,:] = temp_v

        if check_convergence(U_old, U, V_old, V):
            break
    
    return U,V

@njit    
def predict(U:np.ndarray, V:np.ndarray) -> np.ndarray:
        """Predict the entire drug-side effect matrix values."""
        
        return predictions(U, V)

## Evaluation Functions

### Calculate AUPR

In [618]:

def aupr(truth: np.ndarray, predictions: np.ndarray) -> float:
    """Get the area under the precision-recall curve."""
    
    max_value = predictions.max()
    min_value = predictions.min()
    # Create an array of 99 representing the thresholds
    threshold = min_value + (max_value-min_value)*(np.arange(1,100,1)/100)
    
    tn = np.zeros((threshold.size, 1))
    tp = np.zeros((threshold.size, 1))
    fn = np.zeros((threshold.size, 1))
    fp = np.zeros((threshold.size, 1))
    
    # Calculate the tp, tn, fp, fn for every threshold.
    for index in range(99):
        tp[index,0] = np.logical_and(predictions>=threshold[index], truth==1).sum()
        tn[index,0] = np.logical_and(predictions<threshold[index], truth==0).sum()
        fp[index,0] = np.logical_and(predictions>=threshold[index], truth==0).sum()
        fn[index,0] = np.logical_and(predictions<threshold[index], truth==1).sum()
    
    # Calculate the area under the precision-recall curve
    recall = tp/(tp+fn)
    prec = tp/(tp+fp)
    
    x = recall.copy()
    y = prec.copy()
    
#     sorted_index = np.argsort(x, axis=0)
#     
    sorted_index = x.argsort(axis=0)
    y = np.take_along_axis(y, sorted_index, axis=0)
    x = np.take_along_axis(x, sorted_index, axis=0)
    # x = x[sorted_index]
    # y = y[sorted_index]
    

    # Note that Zhang implementation sets the first values for x and y as 0 and 1 respectively
    x[0][0] = 0
    y[0][0] = 1

    # x and y now have 100 values, not 99, causing no errors in trapezoidal rule calculation (indexoutofbound)
    x = np.append(x, [[1]], 0)
    y = np.append(y, [[0]], 0)

    
    # Calculate the area using the trapezoidal rule: (b-a)*0.5*(f(b)+f(a))
    area = 0
    area = 0.5*x[0][0]*(1+y[0][0]) # still 0
    for idx in range(99):
        area += (x[idx+1][0]-x[idx][0])*0.5*(y[idx][0]+ y[idx+1][0])
    
    return area
    



In [619]:
# @njit(cache=True)
# def argsort_pure_python(seq):
#     #http://stackoverflow.com/questions/3382352/equivalent-of-numpy-argsort-in-basic-python/3382369#3382369
#     #by unutbu
#     return sorted(range(len(seq)), key=seq.__getitem__)

@njit(cache=True)
def argsort_pure_python_2(seq):
    # http://stackoverflow.com/questions/3382352/equivalent-of-numpy-argsort-in-basic-python/3383106#3383106
    #lambda version by Tony Veijalainen
#     return [x for x,y in sorted(enumerate(seq), key = lambda x: x[1])]
    result = []
    for x,y in sorted(enumerate(seq), key = lambda x: x[1]):
        result.append(x)
    return result
        
@njit
def argsort_pure_python_3(seq):
    pass
    # http://stackoverflow.com/questions/3382352/equivalent-of-numpy-argsort-in-basic-python/3383106#3383106
    #non-lambda version by Tony Veijalainen
#     return [i for (v, i) in sorted((v, i) for (i, v) in enumerate(seq))]

### Calculate AUROC

In [620]:
@njit
def auroc(truth: np.ndarray, predictions: np.ndarray) -> float:
    """Get the area under the ROC curve."""
    
    max_value = predictions.max()
    min_value = predictions.min()
    # Create an array of 99 representing the thresholds
    threshold = min_value + (max_value-min_value)*(np.arange(1,100,1)/100)
    
    tn = np.zeros((threshold.size, 1))
    tp = np.zeros((threshold.size, 1))
    fn = np.zeros((threshold.size, 1))
    fp = np.zeros((threshold.size, 1))
    
    # Calculate the tp, tn, fp, fn for every threshold.
    for index in range(99):
        tp[index,0] = np.logical_and(predictions>=threshold[index], truth==1).sum()
        tn[index,0] = np.logical_and(predictions<threshold[index], truth==0).sum()
        fp[index,0] = np.logical_and(predictions>=threshold[index], truth==0).sum()
        fn[index,0] = np.logical_and(predictions<threshold[index], truth==1).sum()
    
    # Calculate the area under the precision-recall curve
    sn = tp/(tp+fn)
    sp = tn/(tn+fp)
    x = 1 - sp
    y = sn.copy()
    
    # Zhang sorted twice - redundant
#     sorted_index = x.argsort()
#     x = x[sorted_index]
#     y = y[sorted_index]

#     print("here1")
#     sorted_index = y.argsort(axis=0)
#     y = np.take_along_axis(y, sorted_index, axis=0)
#     x = np.take_along_axis(x, sorted_index, axis=0)
    sorted_index = argsort_pure_python_2(y)
    y = y[sorted_index]
    x = x[sorted_index]
   


    # x and y now have 100 values, not 99, causing no errors in trapezoidal rule calculation (indexoutofbound)
    x = np.append(x, [[1]], 0)
    y = np.append(y, [[1]], 0)

    
    # Calculate the area using the trapezoidal rule: (b-a)*0.5*(f(b)+f(a))
    area = 0
    area = 0.5*x[0][0]*(y[0][0]) # still 0
    for idx in range(99):
        area += (x[idx+1][0]-x[idx][0])*0.5*(y[idx][0]+ y[idx+1][0])
    
    return area

### Calculate the other metrics

In [621]:
@njit(cache=True)
def classification_metric(truth: np.ndarray, predictions: np.ndarray) -> tuple:
    """Calculate the evaulation metrics given 1-D vector of ground truth and predictions."""
    
    tp = np.logical_and(predictions==1, truth==1).sum()
    tn = np.logical_and(predictions==0, truth==0).sum()
    fp = np.logical_and(predictions==1, truth==0).sum()
    fn = np.logical_and(predictions==0, truth==1).sum()
    
    acc = (tp+tn)/(tn+tp+fn+fp)
    sn = tp/(tp+fn)
    recall = sn
    sp = tn/(tn+fp)
    prec = tp/(tp+fp)
    f1 = (2*prec*recall)/(recall+prec)
    
    return sn,sp,prec,acc,f1
    

In [622]:
@njit(cache=True)
def get_metric(truth: np.ndarray, predictions: np.ndarray) -> tuple:
    """Calculate the metrics of the drug-side effect matrix."""

    max_value = predictions.max()
    min_value = predictions.min()
    # Create an 1-D array of 999 threshold values in ascending order
    threshold = min_value + (max_value-min_value)*(np.arange(1,1000,1)/1000)
    
    temp_sn = np.zeros(999)
    temp_sp = np.zeros(999)
    temp_prec = np.zeros(999)
    temp_acc = np.zeros(999)
    temp_f1 = np.zeros(999)
    
    for index in range(999):
        # assign values above threshold to 1
        pred = predictions>threshold[index]
        # calculate the metrics for the predictions under threshold i
        temp_sn[index],temp_sp[index],temp_prec[index],temp_acc[index],temp_f1[index] = classification_metric(truth, pred)
    
    # Get index corresponding to max f1 score (optimal value of prec and recall)
    index_max_f1 = temp_f1.argmax()
    sn = temp_sn[index_max_f1]
    sp = temp_sp[index_max_f1]
    prec = temp_prec[index_max_f1]
    acc = temp_acc[index_max_f1]
    f1 = temp_f1[index_max_f1]
    
    return sn, sp, prec, acc, f1    

### Cross validation for parameter selection
Only calculates the aupr values.

In [623]:


def cross_val_5_param_selection(dataset: np.ndarray, lmbda: float, k: float) -> float:
    """Perform 5 fold CV, given regularization term (lambda) and number of latent features (k).
    
    Args:
        dataset (np.ndarray): The dataset
        lmbda (float): The regularization term
        k (int): Number of latent features
    Returns:
        np.ndarray: Average AUPR over the 5 folds.
    """    
    # print("NOOO")
    # print(len(dataset.nonzero()[1]))
    np.random.shuffle(dataset)
    fold_size = dataset.shape[0]//5
    a = dataset[0:fold_size]
    known_se_a = a.nonzero()
    b = dataset[fold_size:2*fold_size]
    known_se_b = b.nonzero()
    c = dataset[2*fold_size:3*fold_size]
    known_se_c = c.nonzero()
    d = dataset[3*fold_size:4*fold_size]
    known_se_d = d.nonzero()
    e = dataset[4*fold_size:]
    known_se_e = e.nonzero()  
    
    # print("Here11111")
    # print("Here11")

    # print("densities of the each of the 5 parts of the dataset:")
    # len_a = len(a.nonzero()[0])
    # len_b = len(b.nonzero()[0])
    # len_c = len(c.nonzero()[0])
    # len_d = len(d.nonzero()[0])
    # len_e = len(e.nonzero()[0])

    # print("set a:"+ str(len_a))
    # print("set b:"+ str(len_b))
    # print("set c:"+ str(len_c))
    # print("set d:"+ str(len_d))
    # print("set e:"+ str(len_e))
    

#     splits = np.split(dataset,5)
#     a, b, c, d, e = splits[0], splits[1], splits[2], splits[3], splits[4]
    
    
    # First fold: model a
    
    test = a
    train = np.concatenate((b,c,d,e),0)
    # print("Here-again")
    # get the col indices of the nonzero values in train
    known_drug_se = train.nonzero()
    # print(known_drug_se)
    side_fx_index = known_drug_se[1]
    drug_index = known_drug_se[0]+fold_size
    # print(len(side_fx_index))
    # print(len(drug_index))
    # print("Here-again222")
    # print("NOOO")
    U,V = fit(drug_index, side_fx_index, dataset, k, lmbda)
    # print("Here-again3")
    aupr_a = aupr(truth=test, predictions=predict(U,V)[0:fold_size])
    
    
    # print("Here2")

    
    # Second fold: model b
    
    test = b
    known_drug_se = a.nonzero()
    side_fx_index = known_drug_se[1]
    drug_index = known_drug_se[0]
    cde = np.concatenate((c,d,e),axis=0)
    known_drug_se = cde.nonzero()
    side_fx_index = np.concatenate((side_fx_index,known_drug_se[1]), axis=0)
    drug_index = np.concatenate((drug_index,known_drug_se[0]+(2*fold_size)), axis=0)

    # get the col indices of the nonzero values in train
    # side_fx_index = test.nonzero()[1]
    # drug_index = np.arange(fold_size, 2*fold_size)

    U,V = fit(drug_index=drug_index, side_fx_index= side_fx_index, dataset=dataset, k=k, lmbda=lmbda)
    aupr_b = aupr(truth=test, predictions=predict(U,V)[fold_size:2*fold_size])
    
    print("Here3")
    
    # Third fold: model c
    test = c
    ab = np.concatenate((a,b),0)
    known_drug_se = ab.nonzero()
    side_fx_index = known_drug_se[1]
    drug_index = known_drug_se[0]
    cde = np.concatenate((d,e),axis=0)
    known_drug_se = cde.nonzero()
    side_fx_index = np.concatenate((side_fx_index,known_drug_se[1]), axis=0)
    drug_index = np.concatenate((drug_index,known_drug_se[0]+(3*fold_size)), axis=0)


    # get the col indices of the nonzero values in train
    # side_fx_index = test.nonzero()[1]
    # drug_index = np.arange(2*fold_size,3*fold_size)
    U,V = fit(drug_index=drug_index, side_fx_index= side_fx_index, dataset=dataset, k=k, lmbda=lmbda)
    aupr_c = aupr(truth=test, predictions=predict(U,V)[2*fold_size:3*fold_size])
    
   
    # print("Here4")
    # Fourth fold: model d
    
    test = d
    # train = np.concatenate((a,b,c,e),0)
    test = d
    ab = np.concatenate((a,b,c),0)
    known_drug_se = ab.nonzero()
    side_fx_index = known_drug_se[1]
    drug_index = known_drug_se[0]
   
    known_drug_se = e.nonzero()
    side_fx_index = np.concatenate((side_fx_index,known_drug_se[1]), axis=0)
    drug_index = np.concatenate((drug_index,known_drug_se[0]+(4*fold_size)), axis=0)

    # get the col indices of the nonzero values in train
    # side_fx_index = test.nonzero()[1]
    # drug_index = np.arange(3*fold_size,4*fold_size)
    U,V = fit(drug_index=drug_index, side_fx_index= side_fx_index, dataset=dataset, k=k, lmbda=lmbda)
    aupr_d = aupr(truth=test, predictions=predict(U,V)[3*fold_size:4*fold_size])
    
    # print("Here5")
    # Fifth fold: model e
    test = e
    train = np.concatenate((a,b,c,d),0)
    
    # get the col indices of the nonzero values in train
    known_drug_se = train.nonzero()
    # print(known_drug_se)
    side_fx_index = known_drug_se[1]
    drug_index = known_drug_se[0]
    U, V = fit(drug_index=drug_index, side_fx_index= side_fx_index, dataset=dataset, k=k, lmbda=lmbda)
    aupr_e = aupr(truth=test, predictions=predict(U,V)[4*fold_size:])
     
        
    print("Done")
    return (aupr_a + aupr_b + aupr_c + aupr_d + aupr_e)/5
    

### Finding the optimal parameters

In [624]:

def param_selection(dataset: np.ndarray, k_values: np.ndarray, lmda_values: np.ndarray) -> dict:
    """Calculate the AUPR of the model, for each pair of given k and lambda parameter values.
    
    Possible parameter values:
    k_values = [1,5,10,15,20,25,30,35,40,50,100]
    lmda_values = [0.1,1,5,10,15,20]
    
    Args:
        dataset (np.ndarray): The dataset
    Returns:
        dict: Dictionary of parameter combinations and associated aupr score.
    """
    cartesian = [(a, b) for a in k_values for b in lmda_values] # form GeeksForGeeks
        
    # results = dict() # numba.typed.Dict is invoked
    print(cartesian)
    # Do 5 fold CV for each possible combination of lambda and k (product function gives the cartesian product)
    for (k,lmbda) in cartesian:
        print("Trying parameters: (" , k, ", ", lmbda, ")")
        print("Hello World")
        res = cross_val_5_param_selection(dataset=dataset, lmbda=lmbda, k=k)
        print("AUPR: ", res)
        # results[(k,lmbda)] = res
    
    # return results
        
    

### Cross Validation with the optimal parameters
Calculates SN, SP, Precision, Recall, AUPR, F1, AUROC with the optimal parameters.

To Do: UPdate with jit

In [625]:
def cross_val_5(dataset: pd.DataFrame, lmbda: float|int, k: int) -> np.ndarray:
    """Perform 5 fold CV, given regularization term (lambda) and number of latent features (k).
    
    Args:
        dataset (pd.DataFrame): The dataset
        lmbda (float): The regularization term
        k (int): Number of latent features
    Returns:
        np.ndarray: Average of the metrics (sn, sp, prec, recall, acc, f1, aupr, auc) over the 5 folds.
    
    """
    
#     np.random.shuffle(dataset)
    dataset.sample(frac=1)

    fold_size = dataset.shape[0]//5
    
    # Specify the test sets of each fold
    a = dataset.iloc[0:fold_size]
    b = dataset.iloc[fold_size:2*fold_size]
    c = dataset.iloc[2*fold_size:3*fold_size]
    d = dataset.iloc[3*fold_size:4*fold_size]
    e = dataset.iloc[4*fold_size:5*fold_size]   
    
    # model a
    print("First Fold")
    model = SGD_Recommender(k=k, lmbda=lmbda)
    model.fit(dataset.drop(a.index), dataset)
    pred = pd.DataFrame(model.predict(), index=dataset.index, columns=dataset.columns)
    
    aupr_a = aupr(truth=a.values, predictions=pred.iloc[0:fold_size].values)
    auroc_a = auroc(truth=a.values, predictions=pred.iloc[0:fold_size].values)
    sn_a, sp_a, prec_a, acc_a, f1_a = get_metric(truth=a.values, predictions=pred.iloc[0:fold_size].values)
    
    
    # model b
    print("Second Fold")
    model = SGD_Recommender(k=k, lmbda=lmbda)
    model.fit(dataset.drop(b.index), dataset)
    pred = pd.DataFrame(model.predict(), index=dataset.index, columns=dataset.columns)
    
    aupr_b = aupr(truth=b.values, predictions=pred.iloc[fold_size:2*fold_size].values)
    auroc_b = auroc(truth=b.values, predictions=pred.iloc[fold_size:2*fold_size].values)
    sn_b, sp_b, prec_b, acc_b, f1_b = get_metric(truth=b.values, predictions=pred.iloc[fold_size:2*fold_size].values)

    
    # model c
    print("Third Fold")
    model = SGD_Recommender(k=k, lmbda=lmbda)
    model.fit(dataset.drop(c.index), dataset)
    pred = pd.DataFrame(model.predict(), index=dataset.index, columns=dataset.columns)
    
    aupr_c = aupr(truth=c.values, predictions=pred.iloc[2*fold_size:3*fold_size].values)
    auroc_c = auroc(truth=c.values, predictions=pred.iloc[2*fold_size:3*fold_size].values)
    sn_c, sp_c, prec_c, acc_c, f1_c = get_metric(truth=c.values, predictions=pred.iloc[2*fold_size:3*fold_size].values)
    
    # model d
    print("Fourth Fold")
    model = SGD_Recommender(k=k, lmbda=lmbda)
    model.fit(dataset.drop(d.index), dataset)
    pred = pd.DataFrame(model.predict(), index=dataset.index, columns=dataset.columns)
    
    aupr_d = aupr(truth=d.values, predictions=pred.iloc[3*fold_size:4*fold_size].values)
    auroc_d = auroc(truth=d.values, predictions=pred.iloc[3*fold_size:4*fold_size].values)
    sn_d, sp_d, prec_d, acc_d, f1_d = get_metric(truth=d.values, predictions=pred.iloc[3*fold_size:4*fold_size].values)
    
    # model e
    print("Fifth Fold")
    model = SGD_Recommender(k=k, lmbda=lmbda)
    model.fit(dataset.drop(e.index), dataset)
    pred = pd.DataFrame(model.predict(), index=dataset.index, columns=dataset.columns)
    
    aupr_e = aupr(truth=e.values, predictions=pred.iloc[4*fold_size:5*fold_size].values)
    auroc_e = auroc(truth=e.values, predictions=pred.iloc[4*fold_size:5*fold_size].values)
    sn_e, sp_e, prec_e, acc_e, f1_e = get_metric(truth=e.values, predictions=pred.iloc[4*fold_size:5*fold_size].values)
    
    
    # Calculate the average
    aupr_mean = np.mean([aupr_a, aupr_b, aupr_c, aupr_d, aupr_e])
    auroc_mean = np.mean([auroc_a, auroc_b, auroc_c, auroc_d, auroc_e])
    sn_mean = np.mean([sn_a, sn_b, sn_c, sn_d, sn_e])
    sp_mean = np.mean([sp_a, sp_b, sp_c, sp_d, sp_e])
    prec_mean = np.mean([prec_a, prec_b, prec_c, prec_d, prec_e])
    acc_mean = np.mean([acc_a, acc_b, acc_c, acc_d, acc_e])
    f1_mean = np.mean([f1_a, f1_b, f1_c, f1_d, f1_e])
    
    return aupr_mean, auroc_mean, sn_mean, sp_mean, prec_mean, acc_mean, f1_mean
    

### Training with optimal parameters

In [626]:
def train_optimal_params(dataset: pd.DataFrame, lmbda: float|int, k: int) -> np.ndarray:
    """Do 20 runs of 5 fold CV."""
        
    for run in range(20):
        t_aupr, t_auroc, t_sn, t_sp, t_prec, t_acc, t_f1 =  cross_val_5(dataset=dataset, lmbda=lmbda, k=k)
        aupr, auroc, sn, sp, prec, acc, f1 = aupr+t_aupr, auroc+t_auroc, sn+t_sn, sp+t_sp, prec+t_prec, acc+t_acc, f1+t_f1    
    
    # Return the mean of the metrics over the 20 runs
    return np.array([aupr, auroc, sn, sp, prec, acc, f1])/20


## Running Training

### Training Pauwel

In [627]:
pauwel_copy = pauwel.values.copy()
pauwel_copy
num_equal = (pauwel_copy==pauwel.values).sum()
num_equal

1229880

In [628]:
np.random.shuffle(pauwel_copy)
num_eq = (pauwel.values == pauwel_copy).sum()


In [629]:

print(len(pauwel_copy.nonzero()[1]))
np.random.shuffle(pauwel_copy)
fold_size = pauwel_copy.size//5
print(pauwel_copy.size)
print(pauwel_copy.shape)
print(888*1385//5)
fold_size = pauwel_copy.shape[0]//5
print(pauwel_copy.shape[0]//5)
a = pauwel_copy[0:fold_size]
b = pauwel_copy[fold_size:2*fold_size]
c = pauwel_copy[2*fold_size:3*fold_size]
d = pauwel_copy[3*fold_size:4*fold_size]
e = pauwel_copy[4*fold_size:] 

print("densities of the each of the 5 parts of the dataset:")
len_a = len(a.nonzero()[0])
len_b = len(b.nonzero()[0])
len_c = len(c.nonzero()[0])
len_d = len(d.nonzero()[0])
len_e = len(e.nonzero()[0])
print(np.sum([len_a, len_b, len_c, len_d, len_e]))

print("set a:"+ str(len_a))
print("set b:"+ str(len_b))
print("set c:"+ str(len_c))
print("set d:"+ str(len_d))
print("set e:"+ str(len_e))
    

61102
1229880
(888, 1385)
245976
177
densities of the each of the 5 parts of the dataset:
61102
set a:12180
set b:13576
set c:12663
set d:11440
set e:11243


In [630]:
import time
%matplotlib inline

In [632]:
%time param_selection(pauwel.values.copy(), np.array([1.0]), np.array([0.1]))

[(1.0, 0.1)]
Trying parameters: ( 1.0 ,  0.1 )
Hello World
Here3
Done
AUPR:  0.0512394713835454
CPU times: user 39.4 s, sys: 3.2 s, total: 42.6 s
Wall time: 36.7 s


In [ ]:
start = time.time()

print("Finding optimal params.....")
results_1 = param_selection_1(pauwel)
# Get the parameter combination corresponding to highest AUPR
# k, lmda = max(res, key=res.get)

elapsed_time_1 = time.time() - start
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(elapsed_time_1)))


In [ ]:
results_1

In [ ]:
start = time.time()

print("Finding optimal params.....")
results_2 = param_selection_2(pauwel)
# Get the parameter combination corresponding to highest AUPR
# k, lmda = max(res, key=res.get)

elapsed_time_2 = time.time() - start
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(elapsed_time_2)))


In [ ]:
results_2

In [ ]:
start = time.time()

print("Finding optimal params.....")
results_3 = param_selection_3(pauwel)
# Get the parameter combination corresponding to highest AUPR
# k, lmda = max(res, key=res.get)

elapsed_time_3 = time.time() - start
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(elapsed_time_3)))


In [ ]:
start = time.time()

print("Finding optimal params.....")
results_4 = param_selection_4(pauwel)
# Get the parameter combination corresponding to highest AUPR
# k, lmda = max(res, key=res.get)

elapsed_time_4 = time.time() - start
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(elapsed_time_4)))

In [ ]:
start = time.time()

print("Finding optimal params.....")
results_5 = param_selection_5(pauwel)
# Get the parameter combination corresponding to highest AUPR
# k, lmda = max(res, key=res.get)

elapsed_time_5 = time.time() - start
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(elapsed_time_5)))

In [ ]:
# find the optimal number of params

# print the dictionaries



In [ ]:
print("Now training with optimal params.....")

# 20 runs of 5-CV with optimal parameters
res_optimal = train_optimal_params(dataset=pauwel, lmbda=lmda, k=k)
